Выборка разбивается на две части для обучения и для тестирования модели. Данные на обучение и тест обычно делят случайно и равномерно: нужно чтобы и обучающая, и тестовая выборка были похожие на те данные, на которых модель будет использоваться в боевых условиях. 

Более сложные разбиения могут использоваться если в данных есть временная компонента - например, мы строим скоринговую модель и для теста откладываем данные за последние несколько месяцев, а всю остальную выборку используем для обучения. Такое разбиение имитирует реальную работу модели: мы можем обучаться только на данных прошлых кредитов, а работать модель уже будет на данных из будущего. 

Доли общей выборки для обучения и тестирования обычно 70% и 30% соотвественно. Любые разумные числа подходят, если для обучения используется достаточно много данных (обычно больше 50%), но и для тестирования что-то остается (10% и больше).


![alt text](https://drive.google.com/uc?id=1VEzFiadFptULOKgb8yxRrNEXQSYMDa6P)

## План анализа данных (data mining):

  1. Загрузить данные для обучения
  2. Обработать данные перед обучением модели
  3. Обучить модель на обучающей выборке
  4. Загрузить и предобработать данные для тестирования
  5. Провалидировать модель на тестовой выборке

![img1.jpg](https://drive.google.com/uc?id=1rz_ZP6fptNz9s4zVy5e9Vjhfhy_rw3v_)

## 1. Загрузить данные для обучения

**Шаг 1.1. Загружаем библиотеки** 

Библиотека **warnings** отвечает за то, какие предупреждения (warnings) о работе будут выводиться пользователю. 



```
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
```



Для корректной работы с данными в python требуется загрузить специальную библиотеку
**pandas**, программную библиотеку на языке python для обработки и анализа данных. 



```
import pandas as pd # загружаем библиотеку и для простоты обращения в коде называем её сокращенно pd
```



Для корректной работы с графиками в python требуется загрузить специальную библиотеку
**matplotlib**, программную библиотеку на языке python для визуализации данных двумерной и трехмерной графикой.

Графики используются для облегчения интерпретации полученных результатов, а также в качестве иллюстраций в презентациях и отчетах. 

Оснвные методы для построения:
* plot() - графики
* semilogy() - график логарифметический
* hist() - гистограммы



```
import matplotlib.pyplot as plt # загружаем библиотеку и для простоты обращения в коде называем её сокращенно plt
# указываем, чтобы картинки отображались прямо в ноутбуке 
%matplotlib inline 
```



**Шаг 1.2. Загрузим данные**

Для решения задачи мы будем использовать данные. Они состоят из двух частей: часть для обучения и часть для тестирования модели. Загружаем данные с помощие команды !wget. Для того, чтобы игнорировать сообщения в процессе загрузки используем магическую команду %%capture в первой строке.



```
#%%capture
#!wget ссылка

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
   print('User uploaded file «{name}» with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
```



Для данных в формате xlsx (Excel) будем использовать специальную функцию
из библиотеки pandas для загрузки таких данных **read_excel**.

В функции передаем один атрибут: название таблицы с данными.



```
training_data = pd.read_excel('Название_таблицы_с_данными.xlsx') 
```



*Что важно посмотреть после того, как мы загрузили данные?*
- проверить, что данные действительно загрузились
- посмотреть на данные, чтобы удостовериться, что они правильные: колонки имеют те же названия, что и в таблице и т.д.

Для того чтобы это сделать, нужно вызвать от переменной *training_data* метод **head()**, который выводит первые 5 строк таблицы. 

Для вызова метода объекта необходимо сначала написать *имя объекта*, затем поставить *точку*, затем уже написать *название метода*. Обратите внимание, что в конце обязательно ставить скобочки, потому что метод - это функция и в ней есть аргументы, просто в данном случае мы их не передаем, поэтому оставляем поле пустым



```
training_data.head()
```



**Шаг 1.3. Посмотрим на размеры загруженной таблицы**, у которой мы видели только первые 5 строк.

Для этого вызываем поле **shape** у нашей переменной *training_data*. Поле вызывается также как метод, но в конце скобки не ставятся, так как для поля не предусмотрена передача аргументов.  



```
training_data.shape
```



*Что означает первое и второе число?*
(a,b)

Итак, таблица содержит a строк (объектов) и b столбцов (признаков), включая выходной (целевой) признак.

Таблицу проверили, теперь можно приступать к обработке данных.

# 2. Обработать данные перед обучением модели

**Шаг 2.1. Проверяем данные на наличие пропусков и типов переменных**

Начнем с проверки общей информации о данных.
Для того чтобы это сделать, нужно обратиться вызвать у переменной *training_data* метод **info()**.

Напомним, что в конце необходимо поставить скобочки.



```
training_data.info()
```



Анализируем результата выполнения команды:

Цифры в каждой строчке обозначают количество заполненных (*non-null*) значений.

 Если эти цифры в каждой строчке совпадают с числом строк, то в данных нет пропусков и можно двигаться дальше. 

**Шаг 2.2. Работаем с целевой переменной**

> *Кто такая эта переменная целевая?*

Целевая переменная зависит от условий задачи. К примеру если мы должны прогнозировать успеваемость студента, то целевой переменной будет оценка.




```
target_variable_name = 'Целевая_переменная'
```



Нам нужно выделить в отдельную переменную *training_values* столбец из нашей таблицы, который соответствует определенной выше целевой переменной. Для этого мы у таблицы *training_data* в квадратных скобках указываем имя нужного столбца. В нашем случае это имя записано в переменной *target_variable_name*. 



```
training_values = training_data[target_variable_name]
training_values
```



Отделим входные переменные от выходной (целевой), чтобы можно было построить модель предсказания целевой переменной по входным. 
Для это нужно у переменной *training_data* вызвать метод **drop()**. Результат мы записываем в новую переменную *training_points*. После выполнения запроса *training_points* будет содержать исходную таблицу без целевого столбца. 

Обратите внимание, что в данном случае мы передаем два аргумента:
    1. target_variable_name - название столбца цены, который мы ранее записали в эту переменную и теперь хотим удалить из training_data
    2. axis=1 - означает, что мы удаляем столбец, а в случае axis=0 - означает, что мы удаляем строку

training_points = training_data.drop(target_variable_name, axis=1)

Можно посмотреть результаты этих действий, вызвав метод **head()** и поле **shape**, которыми мы пользовались ранее, но сейчас нужно вызывать их от новой переменной *training_points*.



```
training_points.head()

training_points.shape
```



Видно, что столбца действительно нет, а количество строк не изменилось. Данные в 5 первых строках такие же, как были ранее.

**Шаг 2.3 Нормализуем признаки**

Казалось бы, чем больше коэффициент модели, тем сильнее признак влияет на предсказание.

Посмотрим на разброс признаков: некоторые принимают значения порядка $10-100$, а некоторые --- около $0$:




```
data[numeric_feature_columns].describe()
```




Хорошим тоном является нормализация всех признаков. Например, можно привести значения каждого столбца к шкале  [0..1]  с помощью следующего преобразования:

$$
\frac{x - min(x)}{max(x)}
$$

Сделать это можно как ручками, так и с помощью питона:



```
from sklearn.preprocessing import MinMaxScaler

# Создаём инструмент для нормализации признаков.
min_max_scaler = MinMaxScaler()
# Преобразуем признаки (на выходе будет np.ndarray).
X = min_max_scaler.fit_transform(X)
# Преобразуем np.ndarray обратно в pandas таблицу для удобства.
X = pd.DataFrame(X, columns=X.columns)

X.head()
```



##   3. Обучить модель на обучающей выборке

**Шаг 3.1. Выбираем метод, который будем использовать**

Проще всего начать с простых методов. 
Мы воспользуемся двумя методами для построения моделей и сравним их между собой:
* Линейная регрессия *(linear regression)*
* Случайный лес решающих деревьев *(Random Forest)*

На выбор метода для построения модели влияет набор признаков, размер выборки, интуиция про то, какая связь между входными переменными и целевой. Но часто решение принимается исходя из того, какая модель сработала лучше.

Для корректной работы с методами построения моделей в python требуется загрузить специальную библиотеку

**sklearn**

 программную библиотеку на языке python для для машинного обучения и анализа данных.

Мы импортируем два модуля из этой библиотеки:
 * *linear_model* - тут находятся все линейные модели
 * *ensemble* - тут находятся модели на основе ансамблей



```
from sklearn import linear_model, ensemble
```



Прежде чем начать делать ремонт, нужно подготовить инструменты для работы. Аналогично в нашем случае, прежде чем обучать модели, нужно создать их прототипы.  



```
linear_regression_model = linear_model.LinearRegression()   # создаем прототип
linear_regression_model 
```



Теперь случайный лес (RandomForest)



```
random_forest_model = ensemble.RandomForestRegressor(n_estimators=100)
random_forest_model
```



У модели на основе случайного леса больше параметров. Рассмотрим наиболее важные:
* параметр *n_estimators* определяет, сколько деревьев в лесу,
* в параметре *max_depth* устанавливается, какая максимальная **глубина** у дерева,


![](https://drive.google.com/uc?id=1L2iJhzpvWPDiekoBj7fEDFUQKKxTn1GI)

![](https://drive.google.com/uc?id=1nRtxWOKLKh1BzY_aYHHEZE6IWp4nQLSo)

Вот пример, чтобы было понятно:

![](https://drive.google.com/uc?id=1f2llPKcVmoh3bPqecDxlSkbQsJUphjEU)

Так как модель на основе случайного решающего леса сложнее (может выучить не только линейные зависимости), такая модель обычно обучается медленнее.
Кроме этого, на время обучения влияют значения параметров модели. Например, чем больше деревьев в лесе - тем дольше модель будет учиться.

**Шаг 3.2. Обучить модель**

Теперь, когда мы создали прототипы обеих моделей, можем их обучить с помощью обучающей выборки. 

Для этого вызываем метод **fit()** у каждой модели и передаем ему на вход два аргумента: 
таблицу входных признаков и столбец значений целевой переменной - (training_points, training_values)



```
training_values
```




```
linear_regression_model.fit(training_points, training_values)
```



Делаем тоже самое для модели решающего леса.



```
random_forest_model.fit(training_points, training_values)
```



> *Что вообще произошло? Что-то важное?*




Линейная регрессия, подбор коэффициентов a, b

![img.jpg](https://drive.google.com/uc?id=14ECdA7p9MEs6oWiO-3xP5sj7zsiewX5q)

> *Но у признаков то больше, не только признаки "x" и "y"!*

![img123.jpg](https://drive.google.com/uc?id=14wjVlYFDEq4-AxISk9_1bfUZYk6T41vK)

> *А как подбираются все эти w?*

![](https://drive.google.com/uc?id=134mgWuEVOZIMtAXFfE6Dv5zpx3LnO45y)

* Для двух разных моделей в sklearn методы для обучения модели не отличаются.
* Мы получили две обученные модели. 
* Теперь необходимо провалидировать модели на новых тестовых данных. 

## 4. Загрузить и предобработать данные для тестирования

**Шаг 4.1. Загрузим и проанализируем тестовые данные.**

Для данных в формате xlsx (Excel) будем использовать специальную функцию
из библиотеки pandas для загрузки таких данных **read_excel**.

В функции передаем один атрибут: название файла, в котором находится таблица с данными.



```
test_data = pd.read_excel('Название_файла_с_таблицей.xlsx')
```





```
test_data.head()
```



Посмотрим на размеры загруженной таблицы, так как мы видели только 5 строк

Для этого вызываем поле **shape** у нашей переменной *test_data*. Поле вызывается также как метод, но в конце скобки не ставятся (!), так как для поля не предусмотрена передача аргументов.  



```
test_data.shape
```



*Что означает первое и второе число?* 
Таблица содержит a строк (объектов) и b столбцов (признаков), включая выходной (целевой) признак. Также как в учебных данных до обучения.

Таблицу проверили, теперь можно приступать к обработке данных
Действуем аналогично тому, как делали с данными для обучения

Проверим, есть ли в данных пропуски. Для того чтобы это сделать, нужно обратиться вызвать у переменной *test_data* метод **info()**.



```
test_data.info()
```



Цифры в каждой строчке обозначают количество заполненных (*non-null*) значений. Если эти цифры в каждой строчке совпадают с числом строк, то в данных нет пропусков.

**Шаг 4.2. Отделяем целевую переменную**



```
test_values = test_data[target_variable_name]
test_points = test_data.drop(target_variable_name, axis=1)
```



И проверяем результат записанный в test_points



```
test_points.head()
test_points.shape
```



# 5. Провалидировать модель на тестовой выборке

**Шаг 5.1. Сравнение моделей.**

Теперь мы готовы измерить качество, сразу для двух моделей. А затем и сравнить.

*1. Какая модель лучше?*

Получим прогнозы целевой переменной на тестовых данных для модели линейной регрессии м модели случайного леса. 

Для этого вызовем у каждой модели метод **predict()**, в качестве аргумента передадим *test_points*.



```
test_predictions_linear_regression=linear_regression_model.predict(test_points)
test_predictions_linear_regression
```





```
test_predictions_random_forest = random_forest_model.predict(test_points)
test_predictions_random_forest
```



Качество регрессионных моделей оценим двумя способами: 
1. Сравним визуально прогнозы с настоящими ценами (тестовые с предсказанием)
2. Сравним метрики качества

Визуализируем прогноз линейной модели и настоящие значения из тестовой выборки.


In [ ]:
%matplotlib inline 



```
plt.figure(figsize=(7, 7))
plt.scatter(test_values, test_predictions_linear) # рисуем точки, соответствущие парам настоящее значение - прогноз
plt.plot([0, 6 * 10**6], [0, 6 * 10**6]) # рисуем прямую, на которой предсказания и настоящие значения совпадают
plt.xlabel('Настоящее значение', fontsize=20)
plt.ylabel('Предсказанное значение', fontsize=20);
```



Визуализируем прогноз модели случайного леса и настоящие значения из тестовой выборки



```
plt.figure(figsize=(7, 7))
plt.scatter(test_values, test_predictions_random_forest)
plt.plot([0, 6 * 10**6], [0, 6 * 10**6])
plt.xlabel('Настоящее значение', fontsize=20)
plt.ylabel('Предсказанное значение', fontsize=20);
```



Проверим модели с помощью **метрик качества регрессионной модели**

Для корректного подсчета метрик качества модели в python требуется загрузить их из библиотеки **sklearn**. 

Мы используем две метрики качества:
 * *mean_absolute_error* - средняя абсолютная ошибка $|y_i - \hat{y}_i|$
 * *mean_squared_error* - средняя квадратичная ошибка $(y_i - \hat{y}_i)^2$


```
from sklearn.metrics import mean_absolute_error, mean_squared_error
```



Подсчитаем ошибки для линейной модели.

Для этого вызовем методы **mean_absolute_error()** и **mean_squared_error()**. На вход им передается столбец настоящих значений *test_values* и столбец значений, предсказанных моделью линейной регрессии *test_predictions_linear*.



```
mean_absolute_error_linear_model = mean_absolute_error(test_values, test_predictions_linear) 
mean_squared_error_linear_model =  mean_squared_error(test_values, test_predictions_linear)
```



Подсчитаем ошибки для модели случайного леса.

Для этого вызовем методы **mean_absolute_error()** и **mean_squared_error()**. На вход им передается столбец настоящих значений *test_values* и столбец значений, предсказанных моделью линейной регрессии *test_predictions_random_forest*.



```
mean_absolute_error_random_forest_model = mean_absolute_error(test_values, test_predictions_random_forest)
mean_squared_error_random_forest_model = mean_squared_error(test_values, test_predictions_random_forest)
```



Выведем полученные метрики


```
print("MAE: {0:7.2f}, MSE: {1:7.2f} для модели линейной регрессии".format(
        mean_absolute_error_linear_model, 
        np.sqrt(mean_squared_error_linear_model)))

print("MAE: {0:7.2f}, MSE: {1:7.2f} для модели случайного леса".format(
       mean_absolute_error_random_forest_model, 
       mean_squared_error_random_forest_model))
```



# 6. Выявление важных признаков

**Доп.* Какие признаки самые важные**

Ручной перебор признаков невозможен, если их тысячи, кроме того, часто даже эксперты не могут сказать, использование каких признаков в модели даст прирост качества.
Кроме ручного перебора всех признаков и их визуального анализа, можно использовать оценку значимости признаков с помощью обученных моделей. 

Получить оценку важности признаков можно вызвав поле feature_importances_ у модели random_forest_model



```
random_forest_model.feature_importances_
```



Набор цифр, которые мы получили выше, соответствует признакам, на которых мы обучали модель. Посмотреть эти признаки можно, вызвав у *training_points* метод **keys()**



```
training_points.keys()
```



Представим полученные результаты в более удобном табличном виде. Создадим пустую таблицу *feature_importance* с двумя колонками "Название признака", "Важность признака". Для этого воспользуемся функцией **DataFrame()** из библиотеки pandas и передадим ей в качестве аргумента   
*columns = ["Название признака", "Важность признака"]*



```
feature_importance = pd.DataFrame(columns = ['Название признака', 'Важность признака'])
```



Заполним соответствующие колонки значениями, которые мы получили выше. 

```
feature_importance['Название признака'] = training_points.keys()
feature_importance['Важность признака'] = random_forest_model.feature_importances_
```



выведем полученную таблицу


```
feature_importance
```


Мы можем отсортировать значения по важности, вызвав у *feature_importance* метод **sort_values()**.

Обратите внимание, что в данном случае мы передаем два аргумента:

1. by='Важность признака' - название столбца, по которому мы сортируем
2. ascending=False - означает, что мы сортируем по убиыванию, а в случае ascending=True сортировка будет по возростанию


```
feature_importance.sort_values(by='Важность признака', ascending=False)
```



# Обзор результатов

В этом ноутбуке мы научились 
0. Загружать библиотеки, необходимые для работы.
1. Загружать данные для обучения, представленные в формате excel таблицы.
2. Проводить предварительную обработку данных перед построением и использованием модели машинного обучения: смотреть на части таблицы, понимать, какой размер у выборки данных, выделять отдельные столбцы таблицы в новые таблицы.
3. Обучать модель линейной регрессии и решающего леса на обучающей выборке.
4. Валидировать модель на тестовой выборке с помощью кросс-плота для модели и реальных значений, стандартных ошибок модели.